In [26]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [27]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [28]:
def answer_one():
    n=sum(spam_data["target"])
    
    return n*100/len(spam_data.target)

In [29]:
answer_one()

13.406317300789663

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect=CountVectorizer().fit(X_train)
    features=vect.get_feature_names()
    x=0
    ind=0
    for i in features:
        if len(i)>x:
            x=len(i)
            ind=i
    return ind

In [31]:
answer_two()

'com1win150ppmx3age16subscription'

In [32]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect=CountVectorizer().fit(X_train)
    features=vect.get_feature_names()
    vect_Xtrain=vect.transform(X_train)
    vect_Xtest=vect.transform(X_test)
    model= MultinomialNB(alpha=0.1).fit(vect_Xtrain,y_train)
    pred= model.predict(vect_Xtest)
    
    
    return roc_auc_score(y_test,pred)

In [33]:
answer_three()

0.97208121827411165

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect=TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    feature_names = np.array(vect.get_feature_names())
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    sorted_tf_idfs = X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index]
    
    return (pd.Series(sorted_tf_idfs[:20],index=feature_names[sorted_tfidf_index[:20]]),(pd.Series(sorted_tf_idfs[-20:],index=feature_names[sorted_tfidf_index[-20:]][::-1])))

In [35]:
answer_four()

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    0.932702
 havent       0.980166
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         1.000000
 blank        1.000000
 dty

In [36]:
def answer_five():
    vect=TfidfVectorizer(min_df=3).fit(X_train)
    vect_Xtrain=vect.transform(X_train)
    vect_Xtest=vect.transform(X_test)
    
    model=MultinomialNB(alpha=0.1).fit(vect_Xtrain,y_train)
    
    return roc_auc_score(y_test,model.predict(vect_Xtest))

In [37]:
answer_five()

0.94162436548223349

In [38]:
def answer_six():
    return np.mean([len(x) for x in spam_data.text[spam_data["target"]==0]]) ,np.mean([len(x) for x in spam_data.text[spam_data["target"]==1]])

In [39]:
answer_six()

(71.023626943005183, 138.8661311914324)

In [40]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')


In [41]:
from sklearn.svm import SVC

def answer_seven():
    len_Xtrain= [len(i) for i in X_train]
    len_Xtest= [len(i) for i in X_test] 
    
    vect=TfidfVectorizer(min_df=5).fit(X_train)
    vect_Xtrain=vect.transform(X_train)
    vect_Xtest=vect.transform(X_test)
    
    vect_Xtrain=add_feature(vect_Xtrain, len_Xtrain)
    vect_Xtest=add_feature(vect_Xtest, len_Xtest)
    
    model=SVC(C=10000).fit(vect_Xtrain,y_train)
    
    return roc_auc_score(y_test,model.predict(vect_Xtest))

In [42]:
answer_seven()

0.95813668234215565

In [43]:
def answer_eight():
    x=[ sum(el.isnumeric() for el in i)for i in spam_data.text[spam_data.target==1] ]
    y=[ sum(el.isnumeric() for el in i)for i in spam_data.text[spam_data.target==0] ]
    return np.mean(y),np.mean(x)

In [44]:
answer_eight()

(0.29927461139896372, 15.76037483266399)

In [45]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    len_Xtrain= [len(i) for i in X_train]
    len_Xtest= [len(i) for i in X_test] 
    
    number_Xtrain=[sum(w.isnumeric() for w in i) for i in X_train]
    number_Xtest=[sum(w.isnumeric() for w in i) for i in X_test]
    
    vect=TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    vect_Xtrain=vect.transform(X_train)
    vect_Xtest=vect.transform(X_test)
    
    vect_Xtrain=add_feature(vect_Xtrain, len_Xtrain)
    vect_Xtest=add_feature(vect_Xtest, len_Xtest)
    
    vect_Xtrain=add_feature(vect_Xtrain,number_Xtrain)
    vect_Xtest=add_feature(vect_Xtest,number_Xtest)
    
    model=LogisticRegression(C=100).fit(vect_Xtrain,y_train)
    
    
    
    
    
    return roc_auc_score(y_test,model.predict(vect_Xtest))

In [46]:
answer_nine()

0.96533283533945646

In [47]:
def answer_ten():
    
    
    return (np.mean(spam_data.loc[spam_data['target']==0,'text'].str.count('\W')), 
            np.mean(spam_data.loc[spam_data['target']==1,'text'].str.count('\W')))

In [48]:
answer_ten()

(17.291813471502589, 29.041499330655956)

In [65]:
def answer_eleven():
    len_Xtrain= [len(i) for i in X_train]
    len_Xtest= [len(i) for i in X_test] 
    
    number_Xtrain=[sum(w.isnumeric() for w in i) for i in X_train]
    number_Xtest=[sum(w.isnumeric() for w in i) for i in X_test]
    
    word_Xtrain=X_train.str.count("\W")
    word_Xtest=X_test.str.count("\W")
    
    vect=TfidfVectorizer(min_df=5, ngram_range=(1,3),analyzer='char_wb').fit(X_train)
    vect_Xtrain=vect.transform(X_train)
    vect_Xtest=vect.transform(X_test)
    
    vect_Xtrain=add_feature(vect_Xtrain, len_Xtrain)
    vect_Xtest=add_feature(vect_Xtest, len_Xtest)
    
    vect_Xtrain=add_feature(vect_Xtrain,number_Xtrain)
    vect_Xtest=add_feature(vect_Xtest,number_Xtest)
    
    vect_Xtrain=add_feature(vect_Xtrain,word_Xtrain)
    vect_Xtest=add_feature(vect_Xtest,word_Xtest)
    
    model=LogisticRegression(C=100).fit(vect_Xtrain,y_train)
    
    feature=np.array(vect.get_feature_names()+['length_of_doc', 'digit_count', 'non_word_char_count'])
    
    index=model.coef_[0].argsort()
    
    
    return roc_auc_score(y_test,model.predict(vect_Xtest)),list(feature[index[:10]]),list(feature[index[-10:]][::-1])

In [66]:
answer_eleven()

(0.96616895574079431,
 [' i', 'i ', '.', 'ca', ';', 'my ', 'pe', ' go', 'if', 'n '],
 ['/', '0', '**', 'x', 'ww', 'uk', '***', 'mob', 'ree', '>'])